In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive/dataset'

/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive/dataset


In [3]:
!mkdir -p raw/quora raw/parabank neg indices

In [16]:
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv -P raw/quora
!wget https://storage.googleapis.com/paws/english/paws_qqp.tar.gz -P raw/quora
!tar -xf raw/quora/paws_qqp.tar.gz -C raw/quora

--2022-08-24 10:02:34--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.1.2, 151.101.65.2, 151.101.129.2, ...
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.1.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58176133 (55M) [text/tab-separated-values]
Saving to: ‘raw/quora/quora_duplicate_questions.tsv’

quora_duplicate_que 100%[===================>]  55.48M  51.5MB/s    in 1.1s    

2022-08-24 10:02:35 (51.5 MB/s) - ‘raw/quora/quora_duplicate_questions.tsv’ saved [58176133/58176133]

--2022-08-24 10:02:36--  https://storage.googleapis.com/paws/english/paws_qqp.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 142.250.141.128, 2607:f8b0:4023:c0b::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322466 (315K) [application/gzip]
Saving

In [17]:
!pip install nltk==3.2.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
!python qqp_generate_data.py \
  --original_qqp_input="raw/quora/quora_duplicate_questions.tsv" \
  --paws_input="raw/quora/paws_qqp/train.tsv" \
  --paws_output="raw/train_quora.tsv"

!python qqp_generate_data.py \
  --original_qqp_input="raw/quora/quora_duplicate_questions.tsv" \
  --paws_input="raw/quora/paws_qqp/dev_and_test.tsv" \
  --paws_output="raw/dev_and_test_quora.tsv"

Reading the original QQP corpus. This may take a while.
Generating the PAWS-QQP corpus.
******Final Results********
  Success: 11988
  Fail:    0
  Total:   11988
Reading the original QQP corpus. This may take a while.
Generating the PAWS-QQP corpus.
******Final Results********
  Success: 677
  Fail:    0
  Total:   677


In [19]:
def para(row, anchor_col="sentence1", para_col="sentence2", label_col="label", path="paws", name="quora", split="train"):
  out_row = {
             "sentence1": row[anchor_col][2:-1],
             "sentence2": row[para_col][2:-1],
             "label": row[label_col],
             "path": path,
             "name": name,
             "split": split}
  return out_row

In [20]:
import pandas as pd

In [21]:
df = pd.read_csv("raw/train_quora.tsv", sep="\t")

#df = df[df["label"] == 1]
out_list = []
for idx, row in df.iterrows():
    out_list.append(para(row, split="train"))

pd.DataFrame(out_list).reset_index().to_csv("raw/out_paws_quora_train_supervised.csv", index=False)

In [22]:
df = pd.read_csv("raw/dev_and_test_quora.tsv", sep="\t")

#df = df[df["label"] == 1]
out_list = []
for idx, row in df.iterrows():
    out_list.append(para(row, split="test"))

pd.DataFrame(out_list).reset_index().to_csv("raw/out_paws_quora_test_supervised.csv", index=False)